In [8]:
pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 977.2 kB/s eta 0:00:00m eta 0:00:010:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import apache_beam as beam

In [2]:
# Set the logging level to reduce verbose information
import logging

logging.root.setLevel(logging.ERROR)

In [3]:
!head data/SMSSpamCollection


ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham	U dun say so early hor... U c already then say...
ham	Nah I don't think he goes to usf, he lives around here though
spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv
ham	Even my brother is not like to speak with me. They treat me like aids patent.
ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
spam	H

In [4]:
import apache_beam as beam
import re

inputs_pattern = 'data/SMSSpamCollection'
outputs_prefix_ham = 'outputs/ham'
outputs_prefix_spam = 'outputs/spam'

# Ham Word Count
with beam.Pipeline() as pipeline:
     ham = (
      pipeline
      | 'Take in Dataset' >> beam.io.ReadFromText(inputs_pattern)
      | 'Separate to list' >> beam.Map(lambda line: line.split("\t"))
      | 'Keep only ham' >> beam.Filter(lambda line: line[0] == "ham")
      | 'Find words' >> beam.FlatMap(lambda line: re.findall(r"[a-zA-Z']+", line[1]))
      | 'Pair words with 1' >> beam.Map(lambda word: (word, 1))
      | 'Group and sum' >> beam.CombinePerKey(sum)
      | 'Format results' >> beam.Map(lambda word_c: str(word_c))
      | 'Write results' >> beam.io.WriteToText(outputs_prefix_ham, file_name_suffix = ".txt")
    )

# Spam Word Count
with beam.Pipeline() as pipeline1:
  spam = (
    pipeline1
    | 'Take in Dataset' >> beam.io.ReadFromText(inputs_pattern)
    | 'Separate to list' >> beam.Map(lambda line: line.split("\t"))
    | 'Filter out only spam' >> beam.Filter(lambda line: line[0] == "spam")
    | 'Find words' >> beam.FlatMap(lambda line: re.findall(r"[a-zA-Z']+", line[1]))
    | 'Pair words with 1' >> beam.Map(lambda word: (word, 1))
    | 'Group and sum' >> beam.CombinePerKey(sum)
    | 'Format results' >> beam.Map(lambda word_c: str(word_c))
    | 'Write results' >> beam.io.WriteToText(outputs_prefix_spam, file_name_suffix = ".txt")
    )

print('Ham Word Count Head')
! head outputs/ham*.txt

print('Spam Word Count Head')
! head outputs/spam*.txt

Ham Word Count Head
('Go', 10)
('until', 21)
('jurong', 1)
('point', 12)
('crazy', 9)
('Available', 1)
('only', 118)
('in', 770)
('bugis', 6)
('n', 139)
Spam Word Count Head
('Free', 43)
('entry', 25)
('in', 77)
('a', 367)
('wkly', 10)
('comp', 10)
('to', 611)
('win', 38)
('FA', 4)
('Cup', 3)


In [25]:
import apache_beam.runners.interactive.interactive_beam as ib
ib.show_graph(pipeline)